In [ ]:
# Compare the tracking results of different levels of video compression


In [1]:
import os
import glob
import pandas as pd 
import numpy as np

In [2]:
true_labeled_csv = '/home/yiting/Documents/Jarvis_projects/Datasets/annotations/compression_comparison_video/2024-02-25/output_comp0/annotations.csv'
df1 = pd.read_csv(true_labeled_csv, on_bad_lines='skip', header = None, index_col=0) 
df2 = pd.read_csv(true_labeled_csv, skiprows=4, header = None, index_col=0) 
last_column = df2.shape[1]
df2 = df2.drop(columns=[last_column],axis='columns')
df_true = pd.concat([df1,df2])

In [20]:
isbodyparts = [x == 'bodyparts' for x in df_true.index.values]
bodyparts = np.unique(df_true.iloc[isbodyparts].values[0])
bd_idx = 1
# for bd_idx, bd in enumerate(bodyparts):



In [3]:
compressed_folder = r"/home/yiting/Documents/LP_projects/LP_240220/predictions"
compressed_labeled_csvs = [filename for filename in os.listdir(compressed_folder) if filename.endswith('.csv')]


In [4]:
csv = compressed_labeled_csvs[0]
compressed_labeled_csv = os.path.join(compressed_folder,csv)
df_compressed = pd.read_csv(compressed_labeled_csv, header = None, index_col=0) 

In [24]:
csv

'output_comp0_heatmap_mhcrnn_temporal_5.0_pca_singleview_5.0.csv'

In [5]:
frame_names = [os.path.splitext(framename)[0] for framename in df_true.index.values[4:]]
frame_idxes = [int(frame_name[6:])for frame_name in frame_names]
frame_idxes

[980,
 490,
 220,
 570,
 260,
 720,
 620,
 680,
 760,
 310,
 650,
 190,
 380,
 790,
 730,
 740,
 70,
 210,
 710,
 600]

In [21]:
errors = np.full([len(frame_idxes), len(bodyparts)],np.nan)
for k, frame_idx in enumerate(frame_idxes):
    true_row_idx = k + 4
    compressed_row_idx = frame_idx + 3
    true_x = df_true.iloc[true_row_idx, bd_idx*3]
    true_y = df_true.iloc[true_row_idx, bd_idx*3 + 1]
    compressed_x = float(df_compressed.iloc[compressed_row_idx, bd_idx*3])
    compressed_y = float(df_compressed.iloc[compressed_row_idx, bd_idx*3 + 1])
    v = [abs(compressed_x - true_x), abs(compressed_y - true_y)]
    errors[k, bd_idx] = np.linalg.norm(v)
    

In [23]:

# errors = np.full([len(frame_idxes), len(bodyparts)],np.nan)
errors


array([[         nan, 185.33543536,          nan,          nan,
                 nan,          nan,          nan,          nan,
                 nan,          nan,          nan,          nan],
       [         nan, 227.81752645,          nan,          nan,
                 nan,          nan,          nan,          nan,
                 nan,          nan,          nan,          nan],
       [         nan, 224.36068242,          nan,          nan,
                 nan,          nan,          nan,          nan,
                 nan,          nan,          nan,          nan],
       [         nan, 202.12287529,          nan,          nan,
                 nan,          nan,          nan,          nan,
                 nan,          nan,          nan,          nan],
       [         nan, 220.40688687,          nan,          nan,
                 nan,          nan,          nan,          nan,
                 nan,          nan,          nan,          nan],
       [         nan, 191.55828455,

In [ ]:
true_x = df_true.iloc[true_row_idx, bd_idx*3]
true_y = df_true.iloc[true_row_idx, bd_idx*3 + 1]
compressed_x = float(df_compressed.iloc[compressed_row_idx, bd_idx*3])
compressed_y = float(df_compressed.iloc[compressed_row_idx, bd_idx*3 + 1])
v = [abs(compressed_x - true_x), abs(compressed_y - true_y)]
error = np.linalg.norm(v)
error

In [ ]:
df_compressed

In [ ]:
df_compressed.iloc[3,:]

In [ ]:
for csv in compressed_labeled_csvs:
    compressed_labeled_csv = os.path.join(compressed_folder,csv)
    df_compressed = pd.read_csv(compressed_labeled_csv, header = None, index_col=0) 